## groupBy()

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [3]:
schema = StructType([
    StructField('class_cd', StringType()),
    StructField('school', StringType()),
    StructField('class_std_cnt', IntegerType()),
    StructField('loc', StringType()),
    StructField('school_type', StringType()),
    StructField('teaching_type', StringType()),
    
])
cdf = spark.read.csv('/dataframe/a_class_info.csv',header=True, schema=schema)
cdf.printSchema()
cdf.createOrReplaceTempView('class')

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: integer (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)



groupBy  : 집계함수를 가지고 있는 GroupData 객체를 반환한다.  

GrouopData객체의 집계함수들을 사용해 grouping 된 데이터들의 집계결과를 저장하고 있는 DataFrame을 반환 받을 수 있다.

In [9]:
# 지역별 교육타입별 학생 숫자를 구해보자.

# cdf.groupBy(cdf.loc, cdf.teaching_type).agg(F.sum(cdf.class_std_cnt),
#                                             F.avg("class_std_cnt")).show()

print('''지역내 교육타입별 학생 숫자와 평균을 구해고 별칭을 붙여보자.
 단  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.''')

cdf.groupBy(cdf.loc,cdf.teaching_type) \
    .agg(sum(cdf.class_std_cnt).alias('total'),avg(cdf.class_std_cnt).alias('avg')) \
    .select('*').where(col('total')>=300).show()


print('컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자')

지역내 교육타입별 학생 숫자와 평균을 구해고 별칭을 붙여보자.
 단  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.


+--------+-------------+-----+-----------------+
|     loc|teaching_type|total|              avg|
+--------+-------------+-----+-----------------+
|   Urban|     Standard|  631|24.26923076923077|
|Suburban|     Standard|  433|            21.65|
|   Rural|     Standard|  327|          20.4375|
+--------+-------------+-----+-----------------+

컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자


+--------+-------------+------------------+
|     loc|teaching_type|sum(class_std_cnt)|
+--------+-------------+------------------+
|   Rural| Experimental|               211|
|    null|         null|              null|
|   Urban|     Standard|               631|
|Suburban|     Standard|               433|
|   Rural|     Standard|               327|
|Suburban| Experimental|               284|
|   Urban| Experimental|               275|
+--------+-------------+------------------+

지역별 class 숫자를 계산해보자. 단 지역정보가 없는 데이터는 제외한다.


+--------+-----+
|     loc|count|
+--------+-----+
|   Urban|   37|
|Suburban|   34|
|   Rural|   28|
+--------+-----+

지역별 교육타입별 학생 숫자와 평균을 구해보자. 단 학생 숫자가 300미만인 데이터는 제외한다.


+--------+-------------+------------------+------------------+
|     loc|teaching_type|sum(class_std_cnt)|avg(class_std_cnt)|
+--------+-------------+------------------+------------------+
|   Urban|     Standard|               631| 24.26923076923077|
|Suburban|     Standard|               433|             21.65|
|   Rural|     Standard|               327|           20.4375|
+--------+-------------+------------------+------------------+

컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자


+--------+-------------+-----+-----------------+
|     loc|teaching_type|total|              avg|
+--------+-------------+-----+-----------------+
|   Urban|     Standard|  631|24.26923076923077|
|Suburban|     Standard|  433|            21.65|
|   Rural|     Standard|  327|          20.4375|
+--------+-------------+-----+-----------------+



+--------+-------------+-----+-----------------+---+---+------------------+
|     loc|teaching_type|total|              avg|max|min|            stddev|
+--------+-------------+-----+-----------------+---+---+------------------+
|   Urban|     Standard|  631|24.26923076923077| 31| 17| 4.229020617662603|
|Suburban|     Standard|  433|            21.65| 27| 15|3.1668513334889994|
|   Rural|     Standard|  327|          20.4375| 24| 16|2.1899391163530857|
+--------+-------------+-----+-----------------+---+---+------------------+



In [15]:
# 지역별 class 숫자를 계산해보자. 단 지역정보가 없는 데이터는 제외한다.
cdf.groupBy(cdf.loc).agg(count(cdf.loc).alias('count')).where(col('loc').isNotNull()).show()



+--------+-----+
|     loc|count|
+--------+-----+
|   Urban|   37|
|Suburban|   34|
|   Rural|   28|
+--------+-----+



In [117]:
# 반이 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이를 구해보자
# cdf.groupby(cdf.loc,cdf.class_std_cnt).agg(sum(cdf.class_std_cnt).alias('학생합')).groupBy(col('loc'), col('학생합')).agg(sum(col('학생합'))).show()

# 1. 데이터프레임생성 loc컬럼 널값제거
# 지역별로 그룹바이 해줌
# 반의 개수와 학생총합을 데이터프레임 객체로 만들어줌
base =cdf \
        .where(col('loc').isNotNull()) \
        .groupBy(cdf.loc) \
        .agg(count(cdf.class_cd).alias('cnt'), sum(cdf.class_std_cnt).alias('tot'))
base.show()

# 지역별로 학생수를 볼때 학생수가 가장 낮은곳과 낮은곳을 논리실행계획 : 컬럼객체로만듬
# 그다음 그객체를 collect() 함수로 action하여 실제 값으로 가져옴
min_max_row = base.select(max('cnt'),min('cnt')).collect()



# base 데이터프레임의 cnt 컬럼에서 min_max_row의 실제값으로 있는 가장많은 반의 학생수 와 가장적은 반의 학생수를 가져와서
# select문으로 빼기 연산 수행
base.where(base.cnt.isin(min_max_row[0][0], min_max_row[0][1])) \
    .select(max(col('tot')) - min(col('tot'))).show()


+--------+---+---+
|     loc|cnt|tot|
+--------+---+---+
|   Urban| 37|906|
|Suburban| 34|717|
|   Rural| 28|538|
+--------+---+---+



+---------------------+
|(max(tot) - min(tot))|
+---------------------+
|                  368|
+---------------------+



#### sql

In [5]:
# 지역별 교육타입별 학생 숫자를 구해보자.
spark.sql('''
select loc, teaching_type, sum(class_std_cnt) from class group by teaching_type
                ''').show()
print('''지역내 교육타입별 학생 숫자와 평균을 구해보자. 단  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.''')
print('컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자')

AnalysisException: expression 'class.loc' is neither present in the group by, nor is it an aggregate function. Add to group by or wrap in first() (or first_value) if you don't care which value you get.;
Aggregate [teaching_type#5], [loc#3, teaching_type#5, sum(class_std_cnt#2) AS sum(class_std_cnt)#15L]
+- SubqueryAlias class
   +- View (`class`, [class_cd#0,school#1,class_std_cnt#2,loc#3,school_type#4,teaching_type#5])
      +- Relation [class_cd#0,school#1,class_std_cnt#2,loc#3,school_type#4,teaching_type#5] csv


## orderBy()

In [34]:
print('반 학생 숫자를 기준으로 내림차순 정렬하라')
# cdf.select('*') \
#     .orderBy(cdf.class_std_cnt, ascending=0).show(20)
cdf.select('*') \
    .orderBy(cdf.class_std_cnt.desc()).show(20)
print('loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라')
# cdf.select('*') \
#     .orderBy(cdf.loc,cdf.school, ascending=[1,0]).show(20)
cdf.orderBy(cdf.loc.asc(),cdf.school.desc()).show(20)
print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
cdf.select('*') \
    .orderBy(cdf.school_type.asc_nulls_first()).show(20)

반 학생 숫자를 기준으로 내림차순 정렬하라


+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     18K| GOOBU|           31|   Urban|     Public|     Standard|
|     ZBH| ZOWMK|           30|   Urban|     Public|     Standard|
|     A93| VVTVA|           30|   Urban|     Public| Experimental|
|     YTB| VVTVA|           30|   Urban|     Public| Experimental|
|     Q0E| ZOWMK|           30|   Urban|     Public| Experimental|
|     QA2| ZOWMK|           30|   Urban|     Public|     Standard|
|     7BL| VVTVA|           29|   Urban|     Public|     Standard|
|     SUR| QOQTS|           28|   Urban|     Public|     Standard|
|     1Q1| CUQAM|           28|   Urban|     Public|     Standard|
|     OMI| CUQAM|           28|   Urban|     Public|     Standard|
|     HKF| GOOBU|           28|   Urban|     Public|     Standard|
|     0N7| QOQTS|           28|   Urban|     Public| Experimen

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     5SD|  null|         null| null|       null|         null|
|     4SZ|  null|         null| null|       null|         null|
|     6PP|  null|         null| null|       null|         null|
|     341| VKWQH|           18|Rural|     Public|     Standard|
|     DFQ| VKWQH|           19|Rural|     Public| Experimental|
|     GYM| VKWQH|           20|Rural|     Public|     Standard|
|     D33| VKWQH|           21|Rural|     Public|     Standard|
|     IEM| VKWQH|           22|Rural|     Public| Experimental|
|     CD8| VHDHF|           20|Rural| Non-public| Experimental|
|     J6X| VHDHF|           16|Rural| Non-public|     Standard|
|     KR1| VHDHF|           15|Rural| Non-public| Experimental|
|     6U9| OJOBU|           22|Rural|     Public|     Standard|
|     FS3| OJOBU|           19|Rural|   

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     5SD|  null|         null|    null|       null|         null|
|     4SZ|  null|         null|    null|       null|         null|
|     6PP|  null|         null|    null|       null|         null|
|     5LQ| FBUMG|           18|   Rural| Non-public| Experimental|
|     JGD| FBUMG|           14|   Rural| Non-public| Experimental|
|     197| FBUMG|           14|   Rural| Non-public| Experimental|
|     PGK| CCAAW|           21|Suburban| Non-public|     Standard|
|     UWK| CCAAW|           19|Suburban| Non-public|     Standard|
|     EID| CIMBB|           21|   Urban| Non-public|     Standard|
|     05H| IDGFP|           22|   Urban| Non-public|     Standard|
|     MDE|  null|           10|   Rural| Non-public| Experimental|
|     98D| IDGFP|           21|   Urban| Non-public| Experimen

#### sql

In [22]:
cdf.printSchema()
print('반 학생 숫자를 기준으로 내림차순 정렬하라')
spark.sql("select * from class order by class_std_cnt desc").show()

print('loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라')
spark.sql("select * from class order by loc asc, school desc").show()


print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
spark.sql("select * from class order by school_type asc nulls last").show()

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: integer (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)

반 학생 숫자를 기준으로 내림차순 정렬하라


+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     18K| GOOBU|           31|   Urban|     Public|     Standard|
|     ZBH| ZOWMK|           30|   Urban|     Public|     Standard|
|     A93| VVTVA|           30|   Urban|     Public| Experimental|
|     YTB| VVTVA|           30|   Urban|     Public| Experimental|
|     Q0E| ZOWMK|           30|   Urban|     Public| Experimental|
|     QA2| ZOWMK|           30|   Urban|     Public|     Standard|
|     7BL| VVTVA|           29|   Urban|     Public|     Standard|
|     SUR| QOQTS|           28|   Urban|     Public|     Standard|
|     1Q1| CUQAM|           28|   Urban|     Public|     Standard|
|     OMI| CUQAM|           28|   Urban|     Public|     Standard|
|     HKF| GOOBU|           28|   Urban|     Public|     Standard|
|     0N7| QOQTS|           28|   Urban|     Public| Experimen

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     5SD|  null|         null| null|       null|         null|
|     4SZ|  null|         null| null|       null|         null|
|     6PP|  null|         null| null|       null|         null|
|     341| VKWQH|           18|Rural|     Public|     Standard|
|     DFQ| VKWQH|           19|Rural|     Public| Experimental|
|     GYM| VKWQH|           20|Rural|     Public|     Standard|
|     D33| VKWQH|           21|Rural|     Public|     Standard|
|     IEM| VKWQH|           22|Rural|     Public| Experimental|
|     CD8| VHDHF|           20|Rural| Non-public| Experimental|
|     J6X| VHDHF|           16|Rural| Non-public|     Standard|
|     KR1| VHDHF|           15|Rural| Non-public| Experimental|
|     6U9| OJOBU|           22|Rural|     Public|     Standard|
|     FS3| OJOBU|           19|Rural|   

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
|     EPS| CCAAW|           20|Suburban| Non-public| Experimental|
|     IQN| CCAAW|           15|Suburban| Non-public| Experimental|
|     PGK| CCAAW|           21|Suburban| Non-public|     Standard|
|     UHU| CCAAW|           16|Suburban| Non-public| Experimental|
|     UWK| CCAAW|           19|Suburban| Non-public|     Standard|
|     A33| CIMBB|           19|   Urban| Non-public|     Standard|
|     EID| CIMBB|           21|   Urban| Non-public|     Standard|
|     HUJ| CIMBB|           17|   Urban| Non-public| Experimental|
|     PC6| CIMBB|           17|   Urban| Non-public|     Stand